## 本程序从解压缩的计算文件中挑选出能够对数据集构建有用的文件（不对配位数等有没有物理意义做选择），如coor、cluster、xmu、chi_FEFF、expWT、expFT和expBFT等，读取其feature/label，转成文件。

### 将本程序安放在“种子文件_single_xyz_WT”（名称可更改）所在文件夹（一级目录）下的任意一个文件夹下（二级目录）。
### 经过解压缩后的文件夹为“AuPd-4000_single_xyz_WT_untar”；则经过选择后的文件夹名称为“_fealab”

last update: 2022.5.27

相对于24，修订了已存在_treat情况的bug问题。增加了对feature/label的读取

contacts: zhaohf@ihep.ac.cn

# 配置环境

## 导入各种模块

In [1]:
import os
import sys
import pandas as pd
import shutil
import datetime

输出导入模板的版本。

In [2]:
print('python version:',sys.version)
print('panda version:',pd.__version__)
#print('shutil version:',shutil.__version__)
#print('datetime version:',datetime.__version__)

python version: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
panda version: 1.4.2


# 设置输入文件夹和输出文件夹

## 设置源文件夹，即经过计算集群计算的文件夹

In [3]:
# 设置源文件夹，即经过计算集群计算的文件夹
dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar'

## 设置目标文件夹

In [4]:
# 构建提取文件夹和目录
dir_work = dir_src + '_fealab'

## 设置运行日志文件名

In [5]:
file_log = 'modify2.log'

## 设置准备提取的feature、label和对应的文件

如设置'chi0' : 'chi_FEFF.dat'，则出现问题如下：
Feature/Label : chi0 of file D:\haifeng\work_20211031\XAS-ML\code\datasets setup\AuPd-4000_au150-0_DW_R0_cncr_WT_untar\100\1\chi_FEFF.dat is unrecognized!

如设置'cn5' : 'coor.dat'，则出现问题如下：
Too many shells : cn5 !

thinkpad X1 yoga 2018 笔记本，Core i7-8550U @1.8GHz， 16GB RAM， 运行150*2个数据，下面所有的输出，共耗时23分24秒（2022.5.22），共6000个文件，5.9GB。

thinkpad X1 yoga 2018 笔记本，Core i7-8550U @1.8GHz， 16GB RAM， 运行150*40个数据，只计算xmu、chi、wt、ft、bft、cn1、cr1，共耗时3小时17分3秒。

In [6]:
feature_pick_dict = {
#                     'chi'  : 'chi_FEFF.dat'
                     'xmu'  : 'xmu.dat'
                    ,'xmu0' : 'xmu.dat'
#                    ,'wt'   : 'expWT.dat'
#                    ,'wtrl' : 'expWT.dat'
#                    ,'wtim' : 'expWT.dat'
#                    ,'ft'   : 'expFT.dat'
#                    ,'ftrl'   : 'expFT.dat'
#                    ,'ftim'   : 'expFT.dat'
#                    ,'bft'  : 'expBFT.dat'
                    }
# cn1 is the coordinate number of first shell
# cr1 is the average bond distance of first shell
label_pick_dict = {
#                   'cn1' : 'coor.dat'
#                  ,'cn2' : 'coor.dat'
#                  ,'cn3' : 'coor.dat'
#                  ,'cn4' : 'coor.dat'
#                  ,'cn'  : 'coor.dat'
#                  ,'cr1' : 'coor.dat'
#                  ,'cr2' : 'coor.dat'
#                  ,'cr3' : 'coor.dat'
#                  ,'cr4' : 'coor.dat'
#                  ,'cr'  : 'coor.dat'
                  }

# 部分函数模块

## 特征读写 

### 对于给定的文件和字符串，找到该字符串第一次在文中出现的行号（即处于第几行）

In [7]:
def find_string_line(path_file, string):
    """
    找到path_file文件中string所在的行号，即处于第几行
    
    """
    exist = os.path.exists(path_file)
    if exist:
        l = len(string)
        fin = open(path_file,'rt')
        i = 0
        find = False
        for line in fin.readlines():
            line = line.strip() 
            i = i+1
            if line[:l] == string:
                find = True
                break
        fin.close()
        return find, i
    else:
        return False, 0

### 读取xmu.dat文件中的“#  omega    e    k    mu    mu0     chi     @#”下的数组

In [8]:
def read_xmu(path_xmu):
    """
    读取xmu.dat中energy、k、mu等的数据
    """
    exist = os.path.exists(path_xmu)
    if exist:
        find, m = find_string_line(path_xmu,'#  omega')
        if not find:
            print()
            print('path_xmu file',path_xmu, 'has no mu')
            print()
            return None
        xmu = pd.read_csv(path_xmu,delim_whitespace=True,header=None, skiprows=list(range(m)))
# 查看第一行的k为0，则去掉
        a = xmu.iloc[0,2]
        if a == 0 :
            xmu = pd.read_csv(path_xmu,delim_whitespace=True,header=None, skiprows=list(range(m+1)))
# xmu.dat 共6列，分别为omega、e、k、mu、mu0.chi
#
#        xmu_data = xmu.values
#        N_e = xmu_data.shape[0]
#        omega = []
#        e = []
#        k = []
#        mu = []
#        mu0 = []
#        chi = []
#        for i in range(N_e):
#            omega.append(xmu_data[i,0])
#            e.append(xmu_data[i,1])
#            k.append(xmu_data[i,2])
#            mu.append(xmu_data[i,3])
#            mu0.append(xmu_data[i,4])
#            chi.append(xmu_data[i,5])
#        return omega,e,k,mu,mu0,chi
        return xmu
    else:
        return None   

### 将数组写入文件

In [9]:
# 不同格式，为了feature
def write_file(data_np_array, path_file):
    """
    """
    fout = open(path_file,'w')
    for i in range(data_np_array.shape[0]):
        for j in range(data_np_array.shape[1]):
            fout.write(f"{data_np_array[i][j]:14.6e}")
        fout.write('\n')
    fout.close()
#
#  不同格式，为了cn和cr
def write_file2(data_np_array, path_file):
    """
    """
    fout = open(path_file,'w')
    for i in range(data_np_array.shape[0]):
        for j in range(data_np_array.shape[1]):
#            fout.write(f"{data_np_array[i][j]:6.3f}")
            fout.write(f"{data_np_array[i][j]}")
        fout.write('\n')
    fout.close()

### feature/label读写主程序

In [10]:
# fealab = feature/label
# frd = file to read
# fwt = file to write
def fealab_pick(fealab, frd, fwt):
    """
    """
#
    if fealab == 'chi':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.values
#
# xmu情况，
    elif fealab == 'xmu':
        fealab_pd = read_xmu(frd)
# xmu.dat 共6列，分别为omega、e、k、mu、mu0.chi
        fealab_np = fealab_pd.iloc[:,[0,3]].values
#
# xmu0情况，
    elif fealab == 'xmu0':
#        omega,e,k,mu,mu0,chi = read_xmu(path_file_feature_src)
        fealab_pd = read_xmu(frd)
# xmu.dat 共6列，分别为omega、e、k、mu、mu0.chi
        fealab_np = fealab_pd.iloc[:,[0,4]].values
#
# wt情况
    elif fealab == 'wt':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.iloc[:,[0,1,4]].values
#
# wt real情况
    elif fealab == 'wtrl':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.iloc[:,[0,1,2]].values
#
# wt imaginary情况
    elif fealab == 'wtim':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.iloc[:,[0,1,3]].values
#
# ft情况
    elif fealab == 'ft':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.iloc[:,[0,3]].values
#
# ft real情况
    elif fealab == 'ftrl':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.iloc[:,[0,1]].values
#
# ft imaginary情况
    elif fealab == 'ftim':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.iloc[:,[0,2]].values
#
# bft情况
    elif fealab == 'bft':
        fealab_pd = pd.read_csv(frd,header=None,delim_whitespace=True)
        fealab_np = fealab_pd.values
#
    elif fealab[:2] == 'cn' or fealab[0:2] == 'cr':
        fealab_pd = pd.read_csv(frd,delim_whitespace=True,header=None, skiprows=[0,1])
#        ord('0') = 48, ord('9') = 57
        if fealab == 'cn':
            fealab_np = fealab_pd.iloc[:,[1]].values
        elif fealab == 'cn1':
            fealab_np = fealab_pd.iloc[[0],[1]].values
        elif fealab == 'cn2':
            fealab_np = fealab_pd.iloc[0:2,[1]].values
        elif fealab == 'cn3':
            fealab_np = fealab_pd.iloc[0:3,[1]].values
        elif fealab == 'cn4':
            fealab_np = fealab_pd.iloc[0:4,[1]].values
        elif fealab == 'cr':
            fealab_np = fealab_pd.iloc[:,[2]].values
        elif fealab == 'cr1':
            fealab_np = fealab_pd.iloc[[0],[2]].values
        elif fealab == 'cr2':
            fealab_np = fealab_pd.iloc[0:2,[2]].values
        elif fealab == 'cr3':
            fealab_np = fealab_pd.iloc[0:3,[2]].values
        elif fealab == 'cr4':
            fealab_np = fealab_pd.iloc[0:4,[2]].values
        else:
            print(f" Too many shells : {label} !")
        write_file2(fealab_np, fwt)
        return
# 其他情况
    else:
        print(f" Feature/Label : {fealab} of file {frd} is unrecognized!")
#
    write_file(fealab_np, fwt)
    return
#

# 程序正文

## 确定当前目录和工作目录

In [11]:
# 当前目录
path = os.path.abspath('./')
path

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\jupyterbook'

In [12]:
# 上一级目录
path_par = os.path.abspath(os.path.pardir)
path_par

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup'

## 源文件夹

In [13]:
# 源文件夹
path_src = os.path.join(path_par, dir_src)

In [14]:
# 检查源文件夹是否存在，存在则列出其内部文件夹，不存在则打印错误
exist = os.path.exists(path_src)
if exist:
    list_seed = os.listdir(path_src)
else:
    print()
    print('** Error!! WT untared directory', dir_src, 'is not found!' )
    print()
#    sys.exit()
list_seed

['100',
 '1000',
 '1100',
 '1200',
 '1300',
 '1400',
 '1500',
 '1600',
 '1700',
 '1800',
 '1900',
 '200',
 '2000',
 '2100',
 '2200',
 '2300',
 '2400',
 '2500',
 '2600',
 '2700',
 '2800',
 '2900',
 '300',
 '3000',
 '3100',
 '3200',
 '3300',
 '3400',
 '3500',
 '3600',
 '3700',
 '3800',
 '3900',
 '400',
 '4000',
 '500',
 '600',
 '700',
 '800',
 '900']

In [15]:
# 去除list_seed中不是文件夹的部分
file_del = []
for dir_seed in list_seed:
    path_seed_src = os.path.join(path_src, dir_seed)
    if os.path.isfile(path_seed_src):
        file_del.append(dir_seed)
for file in file_del:
    list_seed.remove(file)
list_seed

['100',
 '1000',
 '1100',
 '1200',
 '1300',
 '1400',
 '1500',
 '1600',
 '1700',
 '1800',
 '1900',
 '200',
 '2000',
 '2100',
 '2200',
 '2300',
 '2400',
 '2500',
 '2600',
 '2700',
 '2800',
 '2900',
 '300',
 '3000',
 '3100',
 '3200',
 '3300',
 '3400',
 '3500',
 '3600',
 '3700',
 '3800',
 '3900',
 '400',
 '4000',
 '500',
 '600',
 '700',
 '800',
 '900']

## 目标文件夹

In [16]:
# 目标文件夹
path_work = os.path.join(path_par, dir_work)
os.makedirs(path_work, exist_ok=True)
path_work

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab'

## 创建日志文件

In [17]:
#file_log = 'modify.log'
path_file_log = os.path.join(path_work, file_log)
flog = open(path_file_log, 'w')

## feature和label 提取

In [18]:
starttime = datetime.datetime.now()
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print()
print(file=flog)
#
for id, dir_seed in enumerate(list_seed):
    starttime1 = datetime.datetime.now()
    print(f" Start seed : {dir_seed}, {id+1} / {len(list_seed)} : {starttime1}")
    print(f" Start seed : {dir_seed}, {id+1} / {len(list_seed)} : {starttime1}", file=flog)
#  在新文件夹内，新建与源一致的文件夹
    path_seed_work = os.path.join(path_work, dir_seed)
    os.makedirs(path_seed_work, exist_ok=True)
#
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 源seed中各个xyz文件夹  
    list_xyz = os.listdir(path_seed_src)
#
# 针对不同feature
    for feature in feature_pick_dict.keys():
#  在每个seed文件夹下，构建feature
        dir_feature_work = feature
        path_dir_feature_work = os.path.join(path_seed_work, dir_feature_work)
#
#  检查是否已经存在feature文件夹，若有，删除其中文件；若没有则构建
        os.makedirs(path_dir_feature_work, exist_ok=True)
        shutil.rmtree(path_dir_feature_work)
        os.makedirs(path_dir_feature_work, exist_ok=True)
#
        for dir_xyz in list_xyz:
#
# 源xyz的路径
            path_xyz_src = os.path.join(path_seed_src, dir_xyz)
# 查找xyz路径（文件夹）下是否存在file_feature_src（如“xmu.dat”）文件
            file_feature_src = feature_pick_dict[feature]
#
            path_file_feature_src = os.path.join(path_xyz_src, file_feature_src)
            exist = os.path.exists(path_file_feature_src)
            if exist:
                result = os.path.splitext(file_feature_src)
                file = feature + '_' + dir_xyz + result[1]
                path_file_feature_work = os.path.join(path_dir_feature_work, file)
#
                fealab_pick(feature, path_file_feature_src, path_file_feature_work)
            else:
                print()
                print(file=flog)
                print(f"  Error! No correspoing feature file {file_feature_src} for {dir_seed} / {dir_xyz}")
                print(f"  Error! No correspoing feature file {file_feature_src} for {dir_seed} / {dir_xyz}", file=flog)
                print()
                print(file=flog)
#
#
# 删除 空文件夹
#
#    try:
#        os.removedirs(path_dir_feature_work)
#    except:
#        pass
        if os.path.exists(path_dir_feature_work):
            if len(os.listdir(path_dir_feature_work)) == 0:
                os.rmdir(path_dir_feature_work)
#
# 针对不同label
    for label in label_pick_dict.keys():
#  在每个seed文件夹下，构建label
        dir_label_work = label
        path_dir_label_work = os.path.join(path_seed_work, dir_label_work)
#
#  检查是否已经存在label文件夹，若有，删除其中文件；若没有则构建
        os.makedirs(path_dir_label_work, exist_ok=True)
        shutil.rmtree(path_dir_label_work)
        os.makedirs(path_dir_label_work, exist_ok=True)
#
        for dir_xyz in list_xyz:
#
# 源xyz的路径
            path_xyz_src = os.path.join(path_seed_src, dir_xyz)
# 查找xyz路径（文件夹）下是否存在file_feature_src（如“xmu.dat”）文件
            file_label_src = label_pick_dict[label]
#
            path_file_label_src = os.path.join(path_xyz_src, file_label_src)
            exist = os.path.exists(path_file_label_src)
            if exist:
                result = os.path.splitext(file_label_src)
                file = label + '_' + dir_xyz + result[1]
                path_file_label_work = os.path.join(path_dir_label_work, file)
#
                fealab_pick(label, path_file_label_src, path_file_label_work)
            else:
                print()
                print(file=flog)
                print(f"  Error! No correspoing label file {file_label_src} for {dir_seed} / {dir_xyz}")
                print(f"  Error! No correspoing label file {file_label_src} for {dir_seed} / {dir_xyz}", file=flog)
                print()
                print(file=flog)
#
# 删除 空文件夹
#
#    try:
#        os.removedirs(path_dir_label_work)
#    except:
#        pass
        if os.path.exists(path_dir_label_work):
            if len(os.listdir(path_dir_label_work)) == 0:
                os.rmdir(path_dir_label_work)
#
    endtime1 = datetime.datetime.now()
    print(f" Finish seed : {dir_seed}, {id+1} / {len(list_seed)} : {endtime1}")
    print(f"== Running time : {endtime1-starttime1}")
    print(f" Finish seed : {dir_seed}, {id+1} / {len(list_seed)} : {endtime1}", file=flog)
    print(f"== Running time : {endtime1-starttime1}", file=flog)
#    if  id >= 1: sys.exit(0)
#
#
endtime = datetime.datetime.now()
#
print()
print(file=flog)
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print(f"== End: {endtime}")
print(f"== End: {endtime}", file=flog)
#
print(f"== Running time : {endtime-starttime}")
print(f"== Running time : {endtime-starttime}", file=flog)
flog.close()

== Start: 2022-05-27 23:25:33.749656

 Start seed : 100, 1 / 40 : 2022-05-27 23:25:33.750654
 Finish seed : 100, 1 / 40 : 2022-05-27 23:25:35.958114
== Running time : 0:00:02.207460
 Start seed : 1000, 2 / 40 : 2022-05-27 23:25:35.958114
 Finish seed : 1000, 2 / 40 : 2022-05-27 23:25:38.261583
== Running time : 0:00:02.303469
 Start seed : 1100, 3 / 40 : 2022-05-27 23:25:38.261583
 Finish seed : 1100, 3 / 40 : 2022-05-27 23:25:40.773486
== Running time : 0:00:02.511903
 Start seed : 1200, 4 / 40 : 2022-05-27 23:25:40.773486
 Finish seed : 1200, 4 / 40 : 2022-05-27 23:25:43.059034
== Running time : 0:00:02.285548
 Start seed : 1300, 5 / 40 : 2022-05-27 23:25:43.059982
 Finish seed : 1300, 5 / 40 : 2022-05-27 23:25:45.777083
== Running time : 0:00:02.717101
 Start seed : 1400, 6 / 40 : 2022-05-27 23:25:45.777083
 Finish seed : 1400, 6 / 40 : 2022-05-27 23:25:48.261657
== Running time : 0:00:02.484574
 Start seed : 1500, 7 / 40 : 2022-05-27 23:25:48.262653
 Finish seed : 1500, 7 / 40 : 20